In [1]:
import os, csv
import pickle

import pandas as pd
from lightgbm import LGBMClassifier

from ipywidgets import interact, Dropdown, Select, Button, Output
from IPython.display import display

from feature_engine.creation import CyclicalFeatures

In [2]:
## load saved data and metadata
su_med_data = pd.read_csv("./su_prescribed_meds.csv")

## load trained model
with open("./saved_model.pkl", "rb") as f:
    trained_model = pickle.load(f)
    
interval_disclaimer = pd.read_csv("./med_interval_disclaimer.csv")

In [3]:
SAVE_RESULTS = True
RESULTS_FILE = "./results_stored.csv"
HEADER = ['su_number', 'medication', 'timecode', 'dose', 'predicted_time']

In [4]:
unique_su = sorted(su_med_data["su_number"].unique().tolist())

In [5]:
bf_intervals = pd.date_range(start="07:00:00", end="12:00:00", freq="1T")
ln_intervals = pd.date_range(start="12:15:00", end="16:00:00", freq="1T")
em_intervals = pd.date_range(start="16:15:00", end="20:00:00", freq="1T")
bt_intervals = pd.date_range(start="20:15:00", end="23:45:00", freq="1T")
ot_intervals = pd.date_range(start="00:00:00", end="07:00:00", freq="1T")

bf_df = pd.DataFrame(data=bf_intervals, columns=["datetime"])
ln_df = pd.DataFrame(data=ln_intervals, columns=["datetime"])
em_df = pd.DataFrame(data=em_intervals, columns=["datetime"])
bt_df = pd.DataFrame(data=bt_intervals, columns=["datetime"])
ot_df = pd.DataFrame(data=ot_intervals, columns=["datetime"])

In [6]:
def make_time_cyclical(df, dt_column="datetime"):
    
    cyclic_feats = ["Given hour", "Given min", "Week Day", "Month", "Quarter"]
    
    df["Given hour"] = df[dt_column].apply(lambda x: x.hour)
    df["Given min"] = df[dt_column].apply(lambda x: x.minute)
    
    df["Week Day"] = df[dt_column].apply(lambda x: x.dayofweek)
    df["Month"] = df[dt_column].apply(lambda x: x.month)
    df["Quarter"] = df[dt_column].apply(lambda x: x.quarter)
    df.drop(dt_column, axis=1, inplace=True)
    
    cyclical = CyclicalFeatures(variables=cyclic_feats, drop_original=True)
    df = cyclical.fit_transform(df)
    
    return df

In [7]:
bf_df = make_time_cyclical(bf_df)
ln_df = make_time_cyclical(ln_df)
em_df = make_time_cyclical(em_df)
bt_df = make_time_cyclical(bt_df)
ot_df = make_time_cyclical(ot_df)

In [8]:
timecode_dict = {"BF":[bf_df, bf_intervals], "LN":[ln_df, ln_intervals],
                 "EM":[em_df, em_intervals], "BT":[bt_df, bt_intervals], "OT":[ot_df, ot_intervals]}

In [9]:
su_widget = Dropdown(options = unique_su, description='SU Number')
med_widget = Dropdown(description="Medication")
timecode_widget = Dropdown(description="Time Code")
dose_widget = Select(description="Dose")

def update_med_options(*args):
    med_widget.options = su_med_data[su_med_data["su_number"]==su_widget.value]["Medication Type"].unique().tolist()
def update_time_code_options(*args):
    timecode_widget.options = su_med_data[(su_med_data["su_number"]==su_widget.value) & \
                                          (su_med_data["Medication Type"]==med_widget.value)] \
                                          ["Medication Time Code"].unique().tolist()
def update_dose_options(*args):
    dose_widget.options = su_med_data[(su_med_data["su_number"]==su_widget.value) & \
                                          (su_med_data["Medication Type"]==med_widget.value) & \
                                          (su_med_data["Medication Time Code"]==timecode_widget.value)] \
                                          ["Schedule Dose"].tolist()
su_widget.observe(update_med_options) 
med_widget.observe(update_time_code_options)
timecode_widget.observe(update_dose_options)

@interact(su_number=su_widget, Medication=med_widget, TimeCode=timecode_widget, Dose=dose_widget)
def print_vals(su_number, Medication, TimeCode, Dose):
    print(f"Selected Service User: {su_number}")
    print(f"Selected Medication: {Medication}")
    print(f"Selected TimeCode: {TimeCode}")
    print(f"Selected Dose: {Dose}")

interactive(children=(Dropdown(description='SU Number', options=(1, 3, 6, 8, 10, 13, 14, 15, 17, 18, 22, 23, 2…

In [10]:
def make_data(su, med, timecode, dose):
    su_meds = su_med_data[(su_med_data["su_number"] == su) & (su_med_data["Medication Type"] == med) \
               & (su_med_data["Medication Time Code"] == timecode) & (su_med_data["Schedule Dose"] == dose)]


    su_meds = su_meds.loc[su_meds.index.repeat(len(timecode_dict[timecode][0]))].reset_index(drop=True)
    time_df = timecode_dict[timecode][0]
    data_df = pd.concat([su_meds, time_df], axis=1)
    data_df.drop("su_number", axis=1, inplace=True)

    data_df["Medication Type"] = data_df["Medication Type"].astype('category')
    data_df["Medication Time Code"] = data_df["Medication Time Code"].astype('category')

    data_df["Route"] = data_df["Route"].astype('category')
    data_df["Form"] = data_df["Form"].astype('category')
    return data_df

In [11]:
def check_interval_disclaimer(su, med):
    disc = interval_disclaimer[(interval_disclaimer["Service User ID"]==su) & (interval_disclaimer["Medication Name"]==med)]
    if not disc.empty:
        return True, disc["Medication Time Interval"].iloc[0]
    else:
        return False, None

In [16]:
def predict_time(b):
    data_df = make_data(su=su_widget.value, med=med_widget.value, timecode=timecode_widget.value, dose=dose_widget.value)
    preds = trained_model.predict_proba(data_df)
    predict_time.data = timecode_dict[timecode_widget.value][1][preds[:,0].argmax()].strftime('%d-%m-%Y %X')
    is_disc, disclaimer = check_interval_disclaimer(su=su_widget.value, med=med_widget.value)
    
    if is_disc:
        with output_box:
            print(f"Recommended Time to administer this medicine is: {predict_time.data}")
            print(f"DISCLAIMER: This Patient has an Interval Restriction on this medicine of {disclaimer}")
    else:
        with output_box:
            print(f"Recommended Time to administer this medicine is: {predict_time.data}")
            
    if SAVE_RESULTS:
        
        data = {'su_number': su_widget.value, 'medication': med_widget.value, 'timecode': timecode_widget.value,
                'dose': dose_widget.value, 'predicted_time': predict_time.data}
        
        # check if the CSV file already exists
        if os.path.isfile(RESULTS_FILE):
            # append data to the existing CSV file
            with open(RESULTS_FILE, 'a', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=HEADER)
                writer.writerow(data)
        else:
            # create a new CSV file and write data to it
            with open(RESULTS_FILE, 'w', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=HEADER)
                writer.writeheader()
                writer.writerow(data)

In [17]:
predict_button = Button(
    description='Predict Time',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run button function',
    icon='play'
)
output_box = Output()
display(predict_button, output_box)
predict_button.on_click(predict_time)

Button(button_style='info', description='Predict Time', icon='play', style=ButtonStyle(), tooltip='Run button …

Output()